In [1]:
import random
import numpy as np
import pandas as pd
import pandas.io.sql as psql
import psycopg2 as pg
import matplotlib.pyplot as plt

In [2]:
def randZipf(n, alpha, numSamples): 
    # Calculate Zeta values from 1 to n: 
    tmp = np.power( np.arange(1, n+1), -alpha )
    zeta = np.r_[0.0, np.cumsum(tmp)]
    # Store the translation map: 
    distMap = [x / zeta[-1] for x in zeta]
    # Generate an array of uniform 0-1 pseudo-random values: 
    u = np.random.random(numSamples)
    # bisect them with distMap
    v = np.searchsorted(distMap, u)
    samples = [t-1 for t in v]
    return samples

To generate Zipf distribution, we need at least three parameters: 
1. The number of k (rank)
2. Alpha which is must > 1
3. The sample size 

In our case, we have a diabetes dataset with the 8 number of attributes (categorical attributes) and 8 measures (numerical attributes). The number of rows = 98052

We have two experiment settings: 
1. Missing based on zipf distribution on Attributes
2. Missing based on zipf distribution on Measures 

Before add missing values to the dataset, we generate the ideal-topk views, sorted the Attributes and Measures based on the highest utility score which is the most important one. 

The best attribute/measure will have more missing rather than attributes/measures which have low ranking 

In [3]:
column_rank = list(range(0,8))
rows = 98052 # 98052 rows
cols = 8 # 8 attributes # 8 measures

We used 11 alpha settings = [1.01, 1.03, 1.06, 1.07, 1.1, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]

In [4]:
# The number of sample is 98052*8 for the attributes 
# The number of sample is 98052*8 for the measures
N_sample = rows*cols
a_list =[1.01, 1.03, 1.06, 1.07, 1.1, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]

In [5]:
print("a, column rank, number of missing, percentage")
for a in a_list:
    s = randZipf(cols, a, N_sample)
    unique, counts = np.unique(s, return_counts=True)
    d = dict(zip(unique, counts))
    for i in column_rank:    
        num_missing, percentage = d[i], d[i]/N_sample
        print(a, i+1, num_missing, percentage)
    print("\n")

a, column rank, number of missing, percentage
1.01 1 291491 0.37160256802512953
1.01 2 143985 0.18355693917513155
1.01 3 95968 0.12234324644066413
1.01 4 72114 0.0919333618896096
1.01 5 57140 0.07284400114225105
1.01 6 47489 0.06054058050830172
1.01 7 40771 0.051976247297352424
1.01 8 35458 0.04520305552155999


1.03 1 295380 0.37656039652429324
1.03 2 144736 0.18451433933015135
1.03 3 95095 0.1212303165667197
1.03 4 71049 0.09057566393342308
1.03 5 56601 0.0721568657447069
1.03 6 46604 0.05941235262921715
1.03 7 39852 0.05080467507037082
1.03 8 35099 0.04474539020111777


1.06 1 303042 0.3863281728062661
1.06 2 145428 0.18539652429323217
1.06 3 94561 0.12054955533798392
1.06 4 69970 0.08920011830457308
1.06 5 54981 0.07009163505078937
1.06 6 44847 0.057172469709949825
1.06 7 38240 0.04874964304654673
1.06 8 33347 0.04251188145065883


1.07 1 304922 0.38872486027821973
1.07 2 146106 0.1862608615836495
1.07 3 94092 0.11995165830375719
1.07 4 69295 0.08833960551544079
1.07 5 54350 0.0692

If we see the result above, using alpha = 1.01, the number of missing in each columns will be: 

`
a, column rank, number of missing, percentage
1.01 1 291734 0.3719123526292171
1.01 2 144161 0.1837813099171868
1.01 3 95464 0.12170073022477869
1.01 4 71719 0.091429802553747
1.01 5 57363 0.07312828907110513
1.01 6 47763 0.06088988495900135
1.01 7 40522 0.051658813690694735
1.01 8 35690 0.04549881695426916
`


And using alpha = 4.0, the number of missing in each columns will be: 

`
a, column rank, number of missing, percentage
4.0 1 725361 0.9247146922041366
4.0 2 45188 0.05760718802268184
4.0 3 8886 0.011328172806266064
4.0 4 2795 0.0035631603638885487
4.0 5 1150 0.0014660588259290989
4.0 6 570 0.0007266552441561621
4.0 7 294 0.0003748011259331783
4.0 8 172 0.00021927140700852608
`

The sum number of missing = N_sample and the sum of percentage missing = 100%

In this experiment we used different missing percentage: 
10%, 20%, 30%,..... 90% missing based on Zipf distribution

Let check the real dataset, for instance, dataset with 10% missing on measure based on zipf distribution with alpha = 1.01

In [6]:
# Example count zipf missing from DB
# Example missing zipf on db_10zipf101_missing_measure1 10% missing measures based on Zipf alpha 1.01

conn = pg.connect("dbname=same_len_col_large_experiment_zipf user=postgres password=zenvisage")
db_10zipf101_missing_measure1 = psql.read_sql("SELECT * FROM db_10zipf101_missing_measure1", conn)
db_10zipf101_missing_measure1.drop(db_10zipf101_missing_measure1.columns[[0]], axis=1, inplace=True)
db_10zipf101_missing_measure1.isnull().sum()

race                    0
gender                  0
age                     0
admission_type_id       0
diag_1                  0
insulin                 0
change                  0
readmitted              0
number_emergency      493
number_outpatient     243
number_inpatient      162
number_diagnoses      121
num_procedures         97
num_medications        77
time_in_hospital       70
num_lab_procedures     61
dtype: int64

In [7]:
# Sum all missing values
db_10zipf101_missing_measure1.isnull().sum().sum()

1324

As shown in the result above the number of missing is very small. Then let see if we use the extream setting: 

1. 100% of missing on attributes
2. Zipf distribution with alpha = 4

In [8]:
# Example count zipf missing from DB
# Example missing zipf on db_100zipf400_missing_attr1 100% missing attributes based on Zipf alpha 4

conn = pg.connect("dbname=same_len_col_large_experiment_zipf user=postgres password=zenvisage")
db_100zipf400_missing_attr1 = psql.read_sql("SELECT * FROM db_100zipf400_missing_attr1", conn)
db_100zipf400_missing_attr1.drop(db_100zipf400_missing_attr1.columns[[0]], axis=1, inplace=True)
db_100zipf400_missing_attr1.isnull().sum()

gender                90634
admission_type_id      5657
age                    1122
insulin                 357
race                    145
diag_1                   71
change                   40
readmitted               22
time_in_hospital          0
num_lab_procedures        0
num_procedures            0
num_medications           0
number_outpatient         0
number_emergency          0
number_inpatient          0
number_diagnoses          0
dtype: int64

The first column has the highest number of missing which is 92% of cells will be replaced with NaN. 
However, with this condition we cannot compare between a certain percent missing on random setting and on zipf distribution setting. 

Let's check the sum of missing values from 100% missing based on zipf distribution with alpha = 4

In [9]:
db_100zipf400_missing_attr1.isnull().sum().sum()

98048